In [ ]:
%%writefile mapper.py
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

# Your code for reading stop words here
def extract_stop_word(file_path):
    with open(file_path) as file_data:
        word_list = file_data.read()
        word_list = word_list.split()
        word_list = set(word_list)
    return word_list

stop_word_list = extract_stop_word(path)



for line in sys.stdin:
    try:
        article_id, content = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        continue
    word_list = re.split('\W*\s+\W*', content, flags=re.UNICODE)
    word_list = [word.lower() for word in word_list if word.lower() not in stop_word_list]
    word_set = set(word_list)
    
    total_number_of_terms = len(word_list)
    counts = Counter(word_list)
    
    for word in word_set:
        number_of_word_in_doc = counts[word]
        tf = number_of_word_in_doc/total_number_of_terms
        print('{}\t{:d}\t{:.2f}'.format(word, article_id, tf))

In [ ]:
%%writefile reducer.py
import sys
import numpy as np

current_word = None

for line in sys.stdin:
    try:
        word, article_id, tf = line.split('\t')
        article_id = int(article)
        tf = float(tf)
    except ValueError as e:
        continue
    
    if current_word == word:
        article_dict[article_id] = tf
    else:
        if current_word:
            idf = 1/np.log(1+len(article_dict))
            for article_id, tf in article_dict.items():
                tf_idf = tf*idf
                print('{}\t{:d}\t{:.2f}'.format(current_word, article_id, tf_idf)
            
        article_dict = dict()
        current_word = word
        article_dict[article_id] = tf

if current_word:
    idf = 1/np.log(1+len(article_dict))
    for article_id, tf in article_dict.items():
        tf_idf = tf*idf
        print('{}\t{:d}\t{:.2f}'.format(current_word, article_id, tf_idf)            

In [ ]:
%%bash

OUT_DIR="tf_idf"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming tf_idf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat tf_idf/* | grep -P 'labor\t12\t' | cut -f3